# **Filter Soma And Neurites From Cellmask**

***Prior to this notebook, you should have already run through [1.0_image_setup](1.0_image_setup.ipynb) and [1.1_infer_masks](1.1_infer_masks_from-composite_with_nuc.ipynb), [1.1a_infer_masks](1.1a_infer_masks_from-composite_single_cell.ipynb), or [1.1b_infer_masks](1.1b_infer_masks_from-composite_multiple-cells.ipynb).***

### ➡️ **Input:**
In this workflow, the neuron cellmask segmentation will be used to "infer" the locations of the soma (or cell body) and the neurites as separate objects based on their shape. No intensity images are necessary for this workflow.

### 🍃 **Soma vs Neurites**
Neurons are morphologically complex cell types containing mulitple subcellular regions that each carry out specific functions. One basic type subregion classification discernable in most confocal microscopy images of a neuron is the distinction between the soma (or cell body) and the neurites (e.g., dendrites and axons). Here, the segmentation of the neurites from the soma is included to enable downstream comparisons of organelles between subregions.

Organelle morphology, distribution, and interactions have been shown to be distinct between different functional regions of cells. For example, the morphological differences in mitochondria between the soma and neurites of a neuron show larger mitochondria volume in neurites than in the soma of the same cell [[1](https://pmc.ncbi.nlm.nih.gov/articles/PMC8423436/)]. Golgi have also been shown to differ in distribution throughout a neuron at different stages of neuronal development as the golgi engages in dendritic golgi translocation, eventually leaving golgi outposts during neuronal maturation [[2](https://doi.org/10.1016/j.celrep.2023.112709)]. Moreover, the interactions between ribosomes and the endoplasmic reticulum are more abundant in the soma and dendrites of neurons as compared to that of the axon [[3](https://pmc.ncbi.nlm.nih.gov/articles/PMC8143122/)].

Comparing across distinct subregions enables specific phenotypes underlying cellular physiology to be linked to morphologically distint subregions of cells. The organelle morphology, interactions, and distribution measurements carried out in [Part 2](/notebooks/part_2_quantification/) are carried out and summarized for the entire cell and within each subregion as part of the Organelle Signature Analysis.


-----

### 👣 **Summary of steps**  

➡️ **EXTRACTION**
- **`STEP 1`** - Read in cellmask

    - Determine whether cellmask file contains nucleus mask or not
    - If cellmask file contains nucleus mask, choose the layer without the nucleus mask

**PRE-PROCESSING**
- **`STEP 2`** - Rescale cellmask image
    
    - Rescale the cell mask to half its original size to reduce upcoming computational and memory requirements

- **`STEP 3`** - Determine soma radius

    - Determine range of possible radii
    - Create a saucer using radius in middle of radii range to serve as a footprint
    - Apply an erosion to rescaled mask using the saucer & determine remaining volume
        - If no remaining volume, remove all radii above the used radii from radii range
        - If there is remaining volume, remove all radii below used radii from radii range 
    - Repeat until 1 or 2 values remain in the radii range

**CORE PROCESSING**
- **`STEP 4`** Segment the soma from cellmask

    - Use determined radius to create a new saucer
    - Apply an erosion using the saucer to the original cellmask
    - Dilate from post-erosion remaining values using the saucer
    - Re-mask image with original cellmask segmentation

- **`STEP 5`** Segment neurites from cellmask

    - Multiply original cellmask and the inverse of the newly made soma mask to get neurite mask
    - Remove small outcrops and loose specks

**POST-PROCESSING**
- **`STEP 6`** Clean soma segmentation

    - Multiply original cellmask and the inverse of the newly made neurite mask to get small outcrops
    - Remove loose specks

- **`STEP 7`** Clean neurites segmentation

    - Multiply original cellmask and the inverse of the newly cleaned soma mask to get neurites
    - Remove loose specks

**EXPORT** ➡️
- **`STEP 8`** - Stack masks

    - Stack masks in order of soma, neurites mask

---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`1.0_image_setup`](1.0_image_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
from pathlib import Path
import os, sys
from infer_subc.core.file_io import (list_image_files, 
                                     read_czi_image, 
                                     import_inferred_organelle, 
                                     export_inferred_organelle)
import napari
from napari.utils.notebook_display import nbscreenshot
from skimage.morphology import isotropic_opening, isotropic_dilation, isotropic_erosion, binary_opening, binary_dilation, binary_erosion
from scipy.ndimage import zoom
from infer_subc.core.img import *
from infer_subc.utils.batch import find_segmentation_tiff_files
import pandas as pd

viewer = napari.Viewer()

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `im_type`, `data_root_path`, `in_data_path`, and `out_data_path`.

In [ ]:
#### USER INPUT REQUIRED ###
im_type = ".tiff"
# data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"
# in_data_path = Path("C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/neurites/segmentations")
# out_data_path = Path("C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/neurites/segmentations/out")

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
in_data_path = data_root_path / "neuron_raw_OME"
out_data_path = data_root_path / "neuron_OME_output-20230424"

#### &#x1F3C3; **Run code; no user input required**

In [11]:
# list files in the input folder
img_file_list = list_image_files(in_data_path,im_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":img_file_list})

,Image Name
0,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\neuron_raw_OME\20221027_C2-107_well_1_cell_1_untreated_Linear_unmixing_decon.tiff


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [12]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# load image and metadata
test_img_name = img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# metadata
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']
print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

# open viewer and add images
viewer = napari.Viewer()
for i in list(range(len(channel_names))):
    viewer.add_image(img_data[i],
                     scale=scale,
                     name=f"Channel {i}")
    
cellmask = find_segmentation_tiff_files(file_path, 'cell', out_data_path, '-')[0]

viewer.grid.enabled = True
viewer.reset_view()
print("\nProceed to Napari window to view your selected raw image.")

-----

## **EXTRACTION**

### **`STEP 1` - Read in Cell Mask**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please determine whether the cellmask file has both the nucleus and the cellmask within it.
- `nuc_present`: a True/False statement of whether the cellmask file contains the nucleus mask
If the cellmask does contain the nucleus mask, we will automatically collect only channel 0 (which should be the cellmask)
- `chan`: an int for the channel the cellmask is located in within the cellmask file for files with both the nucleus and cellmask stacked together. If nuc_present is set to False, this value can be ignored

In [ ]:
#### USER INPUT REQUIRED ###
nuc_present = False
chan = 0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block extracts the cell mask from the cell mask and nucleus mask combined image. It will be the only part of the original image used in the rest of this workflow. The cell mask channel is added to the Napari viewer.

In [ ]:
if nuc_present:
    cell_mask = cellmask[chan]
    viewer.add_image(cell_mask, name="Cell Mask", scale=scale)
else:
    cell_mask = cellmask
    viewer.add_image(cell_mask, name="Cell Mask", scale=scale)

-----
## **PRE-PROCESSING**

### **`STEP 2` - Rescale cellmask image**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block rescales the image so that the memory and computational load of determining the radius is reduced. 

In [ ]:
cell_mask_resize = zoom(cell_mask.copy(), (1, 0.5, 0.5))

### **`STEP 3` - Determine Soma Radius**

#### &#x1F6D1; &#x270D; **User Input Required:**

Decide whether to use isotropic or binary methods. Isotropic methods are more memory and time efficient; however, they are more likely to undersegment the soma as compared to binary methods. Binary methods are more likely to oversegment the soma as compared to the isotropic methods, so they should be used if having the initial segment of the neurites included in the soma region is desired.

The binary method uses a saucer-like shape as the footprint; whereas the isotropic method uses a spherical shape as the footprint. The "saucer" is comprised of 3 2-Dimensional disks stacked into a 3-Dimensional space whereas the ball assumes the same number of voxels wide as it is tall and deep.

The radius is used to recreate a saucer or sphere for the actual segmentation of the soma from the neurites in the following step.

In [ ]:
# Options are 'isotropic' or 'binary'
method = 'binary'

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block contains a loop that continues until there are only 1 or 2 values remaining in the radii range. 

First, the block of code creates a range of possible radii. These radii are then sorted through by checking the efficacy of the radius in the middle of the range of possible radii, and removing either all radii above the tested radius value or below the tested radius value from the list. The above process is repeated until there are only 1 or 2 values remaining in the radii range.

The way that the code checks the efficacy of the radius in the range is either developing a version of the "saucer" that will be used to segment the soma and neurites from the cell mask or using an isotropic erosion method to segment the soma from the cellmask via a sphere. The "saucer" is comprised of 3 2-Dimensional disks stacked into a 3-Dimensional space. The top and bottom disk are set to a shorter radius than the center disk. 

The efficacy of the radius is deemed as whether the radius results in an absence of cell mask or not. If there is an absence of cell mask, the radius is deemed too large, and it alongside all radii above it are removed from the possible radii. If there is no absence of cell mask, the radius is deemed too small, and all radii that are below it are removed from the possible radii. This process repeats until there are only 1 or 2 possible radii remaining, at which point, the average radius of the remaining radii is selected, and divided by 2.

In [ ]:
zz, yy, xx = np.shape(cell_mask_resize)
rad_range = [i+1 for i in list(range(yy // 2))]

if method == 'isotropic':
    while len(rad_range) > 2:
        rad = rad_range[((len(rad_range)) // 2)]
        print(f"Trying radius of {rad}")
        if np.array_equal(isotropic_erosion(cell_mask_resize.copy(), rad), np.zeros_like(cell_mask_resize)):
            rad_range = rad_range[:(rad_range.index(rad))]
            print(f"{rad} is too large")
        else:
            rad_range = rad_range[(rad_range.index(rad)+1):]
            print(f"{rad} is too small")
        print(f"{len(rad_range)} possible radii remaining")
elif method == 'binary':
    while len(rad_range) > 2:
        rad = rad_range[((len(rad_range)) // 2)]
        print(f"Trying radius of {rad}")
        edge = disk(rad//4)
        middle = disk(rad)
        w = (np.shape(middle)[0] - np.shape(edge)[0])//2
        edge = np.pad(edge, ((w,w),(w,w)), mode = 'constant', constant_values=0)
        fp = np.stack((edge, middle, edge))
        if np.array_equal(binary_erosion(cell_mask_resize, fp), np.zeros_like(cell_mask_resize)):
            rad_range = rad_range[:(rad_range.index(rad))]
            print(f"{rad} is too large")
        else:
            rad_range = rad_range[(rad_range.index(rad)+1):]
            print(f"{rad} is too small")
        print(f"{len(rad_range)} possible radii remaining")
else:
    raise ValueError(f"method of {method} was given, but only 'isotropic' or 'binary' is allowed.")

if len(rad_range) == 1:
    opti_rad = rad_range[0]//2
elif len(rad_range) == 2:
    opti_rad = (rad_range[0] + rad_range[1])//4

-----
## **CORE-PROCESSING**

### **`STEP 4` - Segmenting the Soma from the Cell Mask**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block uses the radius found in the previous step to segment the soma from the original cell mask.

This is done either by creating a "saucer" that will be used to segment the soma and neurites from the cell mask or applying an isotropic opening and dilation of the cell mask with a sphere. The "saucer" is comprised of 3 2-Dimensional disks stacked into a 3-Dimensional space. The top and bottom disk are set to a shorter radius than the center disk. 

While code in this step is similar to that of step 3, this step uses the enlarged version of the cell mask, not the resized version used for the determination of the optimal radius.

Afterwards, the soma segmentation is added to Napari for visualization.

In [ ]:
if method == 'isotropic':
    neurites_removed = isotropic_opening(cell_mask, opti_rad)
    soma1 = isotropic_dilation(neurites_removed, (opti_rad)) * cell_mask
elif method == 'binary':
    edge = disk(opti_rad//2)
    middle = disk(opti_rad)
    w = (np.shape(middle)[0] - np.shape(edge)[0])//2
    edge = np.pad(edge, ((w,w),(w,w)), mode = 'constant', constant_values=0)
    fp = np.stack((edge, middle, edge))
    neurites_removed = binary_opening(cell_mask, fp)
    soma1 = binary_dilation(neurites_removed, footprint=ball((opti_rad)//2)) * cell_mask
else: 
    raise ValueError(f"method of {method} was given, but only 'isotropic' or 'binary' is allowed.")

viewer.add_image(soma1, scale=scale, name="Initial Soma Segmentation", opacity=0.3, colormap="magenta", blending='additive')

### **`STEP 5` - Segmenting the Neurites from the Cell Mask**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block uses the inverse of the soma found in the previous step to segment the neurites from the original cell mask.

Afterwards, the neurite mask is added to napari for visualization.

In [ ]:
if method == 'isotropic':
    neurites1 = np.invert(soma1.astype(bool), dtype=bool) * cell_mask
    neurites1 = size_filter_linear_size(img=label(neurites1), min_size=(opti_rad*2), method='3D')
elif method == 'binary':
    neurites1 = np.invert(soma1.astype(bool), dtype=bool) * cell_mask
    neurites1 = size_filter_linear_size(img=label(neurites1), min_size=(opti_rad//2), method='3D')
else: 
    raise ValueError(f"method of {method} was given, but only 'isotropic' or 'binary' is allowed.")


viewer.add_image(neurites1, scale=scale, name="Initial Neurite Segmentation", opacity=0.3, colormap="magenta", blending='additive')

-----
## **POST-PROCESSING**

### **`STEP 6` - Cleaning the Soma Segmentation**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block adds the removed parts of the neurites back to the soma mask, then removes anything not connected to the soma mask.

In [ ]:
soma = label(np.invert(neurites1.astype(bool), dtype=bool) * cell_mask)
soma[soma!=np.bincount(np.ravel(soma)[np.ravel(soma)!= 0]).argmax()] = 0
soma = label(soma)

viewer.add_image(soma, scale=scale, name="Cleaned Soma Segmentation", colormap="green", blending='additive')

### **`STEP 7` - Cleaning the Neurites Segmentation**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block adds the removed parts of the soma mask back to the neurites mask, then removes objects below a certain size.

In [ ]:
if method == 'isotropic':
    neurites = np.invert(soma.astype(bool), dtype=bool) * cell_mask
    neurites = size_filter_linear_size(img=label(neurites), min_size=(opti_rad*2), method='3D')
    neurites = label(neurites)
elif method == 'binary':
    neurites = np.invert(soma.astype(bool), dtype=bool) * cell_mask
    neurites = size_filter_linear_size(img=label(neurites), min_size=(opti_rad//2), method='3D')
    neurites = label(neurites)
else: 
    raise ValueError(f"method of {method} was given, but only 'isotropic' or 'binary' is allowed.")


viewer.add_image(neurites, scale=scale, name="Cleaned Neurite Segmentation", colormap="magenta", blending='additive')

-----
## **EXPORT**

### **`STEP 8` - Stacking Soma and Neurites**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block stacks the soma mask and the neurites mask into one numpy array for ease of exporting.

In [ ]:
som_neu = np.stack([soma, neurites])
viewer.add_image(som_neu)

-----
## **SAVING**

## **`Saving` - Save the segmentation output**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block saves the instance segmentation output to the `out_data_path` specified earlier.

In [ ]:
# Saving file
out_file_n = export_inferred_organelle(som_neu, "soma_neurites", meta_dict, out_data_path)
print(f"saved to: {out_data_path}")

-----
-----
## **Define `filter_soma_neurites()` function**
The following code includes an example of how the workflow steps above are combined into one function. This function can be run below to process a single image. It is included in the <mark>batch process notebook</mark> to run the above analysis on multiple cells. 

#### &#x1F3C3; **Run code; no user input required**

### Define Function to Find Radii
find_rad will take a pre-zoomed image and output the radius for the neurites based on this image. Rather than using a while loop, this function acts as a recursive function to improve memory usage and speed of the computational tasks. 

In [ ]:
def find_rad(in_seg: np.ndarray, method: str, rad_range: list = []) -> int:
    if rad_range == []:                             # Setup of rad range
        zz, yy, xx = np.shape(in_seg)                   # Determines width of the image
        rad_range = [i+1 for i in list(range(yy // 4))] # Uses the width of the image to determine the possible radii
                                                        # Must be smaller than half the size of the image
    if len(rad_range) == 1:                         # If the rad range has only 1 value
        return rad_range[0]//2                          # Sets rad range value / 2 as output
    elif len(rad_range) == 2:                       # If the rad range has 2 values
        return rad_range[1]//2                          # Sets the larger rad range value / 2 as output
    
    if method == 'binary':
        rad = rad_range[((len(rad_range)) // 2)]        # Checks middle of rad range
        print(f"Trying radius of {rad}")                # Updates user on progress
        ################################################################################################################################
        # Creates a "Saucer" shape for filtering (see side profile below)
        #
        #    D   Z
        #    E 2 | 001111100
        #    P 1 | 111111111
        #    T 0 | 001111100
        #    H   +----------- Y
        #          012345678
        #            WIDTH
        #
        edge = disk(rad//4)                                                     # Top & Bottom of Saucer
        middle = disk(rad)                                                      # Middle of Saucer
        w = (np.shape(middle)[0] - np.shape(edge)[0])//2                        # Determines distance between edge & middle saucer radii
        edge = np.pad(edge, ((w,w),(w,w)), mode = 'constant', constant_values=0)# Adds emptiness to ensure saucer is equal dimensions
        #                                Saucer components are combined:
        fp = np.stack((edge,            #          0001111000
                    middle,             #          1111111111
                    edge))              #          0001111000   
        ################################################################################################################################
        if np.array_equal((binary_erosion(in_seg, fp)), np.zeros_like(in_seg)): # If nothing remains after erosion
            rad_range = rad_range[:(rad_range.index(rad))]                          # Remove this radius and all larger radii from rad range
            print(f"{rad} is too large")                                            # Status update
        else:                                                                   # If something remains after erosion
            rad_range = rad_range[(rad_range.index(rad)+1):]                        # Remove all smaller radii from rad range
            print(f"{rad} is too small")                                            # Status update
        print(f"{len(rad_range)} possible radii remaining")                     # Status update
    elif method == 'isotropic':
        rad = rad_range[((len(rad_range)) // 2)]
        print(f"Trying radius of {rad}")
        if np.array_equal(isotropic_erosion(in_seg.copy(), rad), np.zeros_like(in_seg)):
            rad_range = rad_range[:(rad_range.index(rad))]
            print(f"{rad} is too large")
        else:
            rad_range = rad_range[(rad_range.index(rad)+1):]
            print(f"{rad} is too small")
        print(f"{len(rad_range)} possible radii remaining")
    return find_rad(in_seg, method, rad_range)                                      # Try a different radius

#### &#x1F3C3; **Run code; no user input required**

### Define Function to Segment Soma & Neurites
filter_soma_neurites takes in a mask segmentation and outputs a stacked numpy array containing the soma and the neurites in different channels.

In [ ]:
def filter_soma_neurites(in_seg: np.ndarray, nuc_present: bool=False, method: str='binary'):
    cell_mask = zoom(in_seg, (1, 0.5, 0.5)) # Make easier on memory
    
    rad = find_rad(cell_mask, method=method) # Find Radius
    print(f"Radius of {rad} found")
    if method == 'binary':
        ################################################################################################################################
        # Creates a "Saucer" shape for filtering (see side profile below)
        #
        #    D   Z
        #    E 2 | 001111100
        #    P 1 | 111111111
        #    T 0 | 001111100
        #    H   +----------- Y
        #          012345678
        #            WIDTH
        #
        edge = disk(rad//2)                                                     # Top & Bottom of Saucer
        middle = disk(rad)                                                      # Middle of Saucer
        w = (np.shape(middle)[0] - np.shape(edge)[0])//2                        # Determines distance between edge & middle saucer radii
        edge = np.pad(edge, ((w,w),(w,w)), mode ='constant', constant_values=0) # Adds emptiness to ensure saucer is equal dimensions
        #                                Saucer components are combined:
        fp = np.stack((edge,            #          0001111000
                    middle,             #          1111111111
                    edge))              #          0001111000                                   
        #################################################################################################################################

        nr1 = binary_opening(in_seg, fp)                                # Opening of original image occurs w/ saucer as footprint
        soma = binary_dilation(nr1, footprint=ball((rad)//2)) * in_seg  # Dilation of opened image occurs, masked by original image
        neurites = np.invert(soma.astype(bool), dtype=bool) * in_seg    # Determination of the neurites from missing values of in_img
        neurites = size_filter_linear_size(img=label(neurites), min_size=(rad//2), method='3D') # Removes small undesired segments
        soma = label(np.invert(neurites.astype(bool), dtype=bool) * in_seg) # Determination of the soma from missing values of in_img in neurites
        soma[soma!=np.bincount(np.ravel(soma)[np.ravel(soma)!= 0]).argmax()] = 0 # Ensures all values of the soma are the same value
        neurites = np.invert(soma.astype(bool), dtype=bool) * in_seg # Redetermines the neurites from missing values of segmented soma
        neurites = size_filter_linear_size(img=label(neurites), min_size=(rad//2), method='3D') # Once again removes small undesired segments
    elif method == 'isotropic':
        nr1 = isotropic_opening(in_seg, rad)
        soma = isotropic_dilation(nr1, (rad)) * in_seg
        neurites = np.invert(soma.astype(bool), dtype=bool) * in_seg
        neurites = size_filter_linear_size(img=label(neurites), min_size=(rad*2), method='3D')
        soma = label(np.invert(neurites.astype(bool), dtype=bool) * in_seg) # Determination of the soma from missing values of in_img in neurites
        soma[soma!=np.bincount(np.ravel(soma)[np.ravel(soma)!= 0]).argmax()] = 0 # Ensures all values of the soma are the same value
        neurites = np.invert(soma.astype(bool), dtype=bool) * in_seg # Redetermines the neurites from missing values of segmented soma
        neurites = size_filter_linear_size(img=label(neurites), min_size=(rad*2), method='3D') # Once again removes small undesired segments
    return np.stack([label(soma), label(neurites)]) #Relabels soma and neurites for outputs and stacks soma into channel 0 and neurites into 1

In [ ]:
somneu = filter_soma_neurites(cell_mask, method=method)

#confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(somneu == som_neu)}")

# adding image to Napari as a new layer
viewer.add_labels(somneu, scale=scale, name="filter_soma_neurites() output")
viewer.grid.enabled = True
viewer.reset_view()

# screenshot viewer
nbscreenshot(viewer, canvas_only = False)